<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Modelisation_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


import pandas as pd
from PIL import Image
from torchvision import transforms

train_df_images = pd.read_csv('train_df_images.csv')

train_df_images.head()

Saving train_df_images.csv to train_df_images (2).csv


,item,name,description,type,category,subcategory,label,image
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,<PIL.JpegImagePlugin.JpegImageFile image mode=...
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,<PIL.JpegImagePlugin.JpegImageFile image mode=...
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,<PIL.JpegImagePlugin.JpegImageFile image mode=...
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,<PIL.JpegImagePlugin.JpegImageFile image mode=...
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,<PIL.JpegImagePlugin.JpegImageFile image mode=...


In [3]:
train_df_images['image'].count()

np.int64(680)

Fill in the Nan images by white images

In [4]:
pd.isna(train_df_images['image'][9])  # True si c’est NaN

True

In [5]:
def fill_NaN_image(image, size, color=(255, 255, 255)):

  if pd.isna(image) == True:
      image = Image.new('RGB', size, color)

  return image

# Test
image = train_df_images['image'][9]
size = (28,28)
print(fill_NaN_image(image, size))

<PIL.Image.Image image mode=RGB size=28x28 at 0x7AF8F84F3150>


In [6]:
size=(28,28)

train_df_images['image'] = train_df_images['image'].apply(lambda x: fill_NaN_image(x, size))

train_df_images['image'].count()

np.int64(6251)

Reshape the images, so that all the images have the same shape

In [7]:
# Reshape images in the dataframe


def reshape_image(image, size):

  """
  Reshaphe an PIL image in the desired size

  Arguments:
    image : ...
    size : tuple

  Returns:
    image : ...
  """
  if image is not None:

    image = image.resize((size[0], size[1]))

    image = image.convert('RGB')

  return image


In [10]:
size = (28,28)
train_df_images['reshaped_image'] = None
train_df_images['reshaped_image'] = train_df_images['image'].apply(lambda x: reshape_image(x, size))

train_df_images.head()

AttributeError: 'str' object has no attribute 'resize'

Data augmentation, as we don't have a lot of data

In [ ]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(size=28, scale=(0.8, 1.0)),  # zoom entre 80% et 100%
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor()
])

nb_augmentation = 3

for i in range(3):
  train_df_photos[f'augmented_images_{i}'] = None
  train_df_photos[f'augmented_images_{i}'] = train_df_photos['reshaped_images'].apply(lambda x: transform(x))

train_df_photos.head()

Modelisation

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential()

# 1st layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))

# 2nd layer
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# 3rd layer
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Flatten
model.add(layers.Flatten())

# 4th layer
model.add(layers.Dense(64, activation='relu'))

# Output layer
model.add(layers.Dense(3, activation='softmax'))

# Compilation
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Visualisation
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 92,867 (362.76 KB)

 Trainable params: 92,867 (362.76 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
X_train = train_df_images['reshaped_images']
y_train = train_df_images['label']

# Training
model.fit(X_train, y_train, epochs=5, batch_size=32)